In [1]:
import json
import time
from flask import Flask, request, jsonify
from influxdb import InfluxDBClient
import paho.mqtt.client as mqtt

app = Flask(__name__)

# # MQTT Broker connection settings
# mqtt_broker = "localhost"
# mqtt_topic_sampling = "plant/sampling_rate"
# mqtt_topic_position = "plant/change_position"

# mqtt_client = mqtt.Client()
# mqtt_client.connect(mqtt_broker)

# # InfluxDB connection settings
# client = InfluxDBClient(host='localhost', port=8086)
# client.switch_database('plant_light_data')

# Endpoint to receive sensor data
@app.route('/sensor_data', methods=['POST'])
def sensor_data():
    try:
        data = request.get_json()
        timestamp = time.time()

        # Prepare data for InfluxDB
        influx_data = [
            {
                "measurement": "light_sensor",
                "tags": {
                    "sensor_id": "plant_ldr"
                },
                "fields": {
                    "sensor_1_value": data["sensor_1_value"],
                    "sensor_2_value": data["sensor_2_value"]
                },
                "time": int(timestamp)
            }
        ]
        # save to local file
        with open('sensor_data.json', 'a') as f:
            json.dump(data, f)
            f.write('\n')
         
        # # Write to InfluxDB
        # client.write_points(influx_data)

        return jsonify({"status": "success"}), 200

    except Exception as e:
        print(f"Error: {str(e)}")
        return jsonify({"status": "error", "message": str(e)}), 500

# # Function to publish MQTT messages
# def publish_mqtt(topic, message):
#     mqtt_client.publish(topic, message)

# Change sampling rate via MQTT
@app.route('/change_sampling_rate', methods=['POST'])
def change_sampling_rate():
    data = request.get_json()
    rate = data.get('rate')
    
    if rate:
        publish_mqtt(mqtt_topic_sampling, str(rate))
        return jsonify({"status": "success", "message": f"Sampling rate updated to {rate}"}), 200
    else:
        return jsonify({"status": "error", "message": "Invalid rate"}), 400

# Change sensor position via MQTT
@app.route('/change_position', methods=['POST'])
def change_position():
    data = request.get_json()
    position = data.get('position')

    if position:
        publish_mqtt(mqtt_topic_position, position)
        return jsonify({"status": "success", "message": f"Position changed to {position}"}), 200
    else:
        return jsonify({"status": "error", "message": "Invalid position"}), 400

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.20.10.3 - - [08/Oct/2024 16:42:57] "POST /sensor_data HTTP/1.1" 200 -
172.20.10.3 - - [08/Oct/2024 16:43:03] "POST /sensor_data HTTP/1.1" 200 -
172.20.10.3 - - [08/Oct/2024 16:43:09] "POST /sensor_data HTTP/1.1" 200 -
172.20.10.3 - - [08/Oct/2024 16:43:15] "POST /sensor_data HTTP/1.1" 200 -
172.20.10.3 - - [08/Oct/2024 16:43:20] "POST /sensor_data HTTP/1.1" 200 -
172.20.10.3 - - [08/Oct/2024 16:43:25] "POST /sensor_data HTTP/1.1" 200 -
172.20.10.3 - - [08/Oct/2024 16:43:31] "POST /sensor_data HTTP/1.1" 200 -
172.20.10.3 - - [08/Oct/2024 16:43:36] "POST /sensor_data HTTP/1.1" 200 -
172.20.10.3 - - [08/Oct/2024 16:43:41] "POST /sensor_data HTTP/1.1" 200 -


: 

### AUTOGENERATED, TO BE TESTED.
### PYTHON CODE FOR PREDICTION.

In [ ]:
from influxdb import InfluxDBClient
import numpy as np
from sklearn.linear_model import LinearRegression

# Connect to InfluxDB
client = InfluxDBClient(host='localhost', port=8086)
client.switch_database('plant_light_data')

# Function to get the last N hours of sunlight data
def get_sensor_data(sensor_id, hours=24):
    query = f'SELECT "sensor_1_value", "sensor_2_value" FROM "light_sensor" WHERE time > now() - {hours}h'
    result = client.query(query)
    return list(result.get_points())

# Prediction function using linear regression
def predict_light(sensor_id, hours=3):
    data = get_sensor_data(sensor_id)
    
    if len(data) == 0:
        return None

    timestamps = np.array([point['time'] for point in data]).reshape(-1, 1)
    light_values = np.array([point['sensor_1_value'] for point in data])

    # Train linear regression model
    model = LinearRegression()
    model.fit(timestamps, light_values)

    # Predict future sunlight for the next N hours
    future_times = np.array([time + (i * 3600) for i in range(1, hours + 1)]).reshape(-1, 1)
    predicted_values = model.predict(future_times)

    return predicted_values

# Example usage of prediction
predicted_light = predict_light("plant_ldr_1", hours=3)
print(f"Predicted light exposure for the next 3 hours: {predicted_light}")
